In [ ]:
# Import packages for calculations
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    await piplite.install("jinja2")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta
import ipywidgets as widgets

%matplotlib widget
pd.options.display.max_columns = None
pd.options.display.max_rows = None

---

# 17 Bullish Momentum Support

Click $>>$ button to run the entire script first

**Buy Signal**
> BUY 10 shares when MOM(14,0) crosses above 0.0 at 15min candle interval using Candlestick chart chart. Enter trade between 00:00 to 23:59

**Sell Signal**
> SELL 10 shares when Close(0) crosses below Pivot point(pp,standard,No) or at stop loss % of 1.00 or target profit % of 2 at 15min candle interval using Candlestick chart chart.

Please feel free to change to any **NSE SYMBOL** to the below list, separated by a comma

In [ ]:
# Symbol to run Backtest on (SIEMENS)
symbols = "SIEMENS"

In [ ]:
# Download historical data for startegy (2months at 1hour)
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "7d"
intervalOfData = "15m"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        # print(text)
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas
    print(apiSymbolIndivi)
    # print(stockPandasTot)

## An MACD line

<img src="https://th.bing.com/th/id/OIP.oQ5UERWqYAcXvBx3mXfcXgHaE_?pid=ImgDet&rs=1" width="700" height="300"/>

### Buy signal
> BUY 10 shares when MOM(14,0) crosses above 0.0 at 15min candle interval using Candlestick chart chart. Enter trade between 00:00 to 23:59


- $MOM$ calculated as for 14 days
- if $MOM[0]$ > $0$ 
    - is $MOM[-1]$ < $0$
    
### Sell signal
> SELL 10 shares when Close(0) crosses below Pivot point(pp,standard,No) or at stop loss % of 1.00 or target profit % of 2 at 15min candle interval using Candlestick chart chart.

If either of the below conditions are met
- Target Signal met when $Pivot$ calculated at (pp,standard,No)
  - $close[0]$ < $Pivot[0]$
    - $close[-1]$ > $Pivot[-1]$
- Stop Loss condition met if $stock.hour[0] < order.buy*0.99$
- Target price condition met if $stock.close[0] > order.buy*1.02$

In [ ]:
# Define and run strategy (Aroon crossover) (interaction)
display(Markdown("# Set the variables to this strategy here"))
display(Markdown("> We will improve UI shortly"))
display(Markdown("If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly"))
display(Markdown(f'{symbol}'))
@widgets.interact(MOMPeriod=widgets.IntSlider(min=1, max=50, step=1, value=14,continuous_update=False),\
                  stopLoss=widgets.FloatSlider(min=0, max=5, step=0.1, value=2,continuous_update=False),\
                  targetPrice=widgets.FloatSlider(min=1, max=8, step=0.1, value=6,continuous_update=False),\
                  numberOfStocks=widgets.IntSlider(min=0, max=100, step=1, value=10,continuous_update=False),\
                  initialCash=widgets.IntSlider(min=0, max=100000, step=100, value=50000,continuous_update=False),
                 )
def voilas(MOMPeriod,stopLoss,targetPrice,numberOfStocks,initialCash):
    class CrossOverStrat(bt.Strategy):
        def __init__(self):
            # Keep a reference to the hourly dataseries
            self.min_15 = self.datas[0]
            #Some indicators
            self.MOM = bt.indicators.Momentum(self.min_15,period=MOMPeriod)
            self.Pivot = bt.indicators.PivotPoint(self.datas[1])
            # To keep track of pending orders and buy price/commission
            self.order = None
            self.buyprice = None
            self.buycomm = None

        def log(self, txt, dt=None):
            ''' Logging function for this strategy'''
            dt = dt or self.datas[0].datetime.datetime()
            print('%s, %s' % (dt, txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                else:  # Sell
                    self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            self.order = None

        def notify_trade(self, trade):
            if not trade.isclosed:
                return

            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

        def next(self):
            # Simply log the closing price of the series from the reference
            # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.order:
                return
            # Check if we are in the market
            if not self.position:
                # Meet conditions of DonchianBreakout
                # self.log("0")
                if self.MOM.momentum[0] >= 0:
                    # self.log("1")
                    if self.MOM.momentum[-1] < 0:
                        # self.log("2")
                        self.order = self.buy(size=numberOfStocks)
     
            if self.position:
                # Already in the market ... we might sell
                if self.min_15.close[0] <= self.Pivot[0]:
                    if self.min_15.close[-1] > self.Pivot[-1]:
                        self.order = self.sell(size=numberOfStocks)
                if self.min_15.close >= (1+targetPrice/100)*self.buyprice:
                    self.log(f'target price reached')
                    self.order = self.sell(size=numberOfStocks)
                if self.min_15.close <= (1-stopLoss/100)*self.buyprice:
                    self.log(f'stop loss reached')
                    self.order = self.sell(size=numberOfStocks)
    
    ##### End of strategy section #####
    cerebro = bt.Cerebro()
    
    #Add the data to the cerebro instance
    data = bt.feeds.PandasData(dataname=stockPandasTot[symbol+".NS"][::-1],timeframe=bt.TimeFrame.Minutes,compression=15)
    cerebro.adddata(data)
    cerebro.resampledata(data,timeframe=bt.TimeFrame.Days)
    #Add the strategy for the backtesting
    cerebro.addstrategy(CrossOverStrat)
    #Set initial cash
    cerebro.broker.setcash(initialCash)
    #A comission typically taken by broker %(value) ##To be worked on for exact calculation
    cerebro.broker.setcommission(commission=0.0025)
    #Add observer for backtesting results
    cerebro.addobserver(bt.observers.BuySell,bardist=0.001,barplot=True)
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.Trades)
    

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run(stdstats=False)

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot(style='candlestick')

---
### Understanding above plot

> Please increase the size of the plot to the entire width of the screen, before reading this section. You can do the same by dragging the triangular element shown at the bottom right corner of the plot

The plot above has quite a lines and legends on them.
The legend is available on the left side indicating various items.

We shall move from the bottom most element and move upwards.
#### Donchian Channel
The Donchian channel is a useful indicator for seeing the volatility of a market price. If a price is stable the Donchian channel will be relatively narrow. If the price fluctuates a lot the Donchian channel will be wider. Its primary use, however, is for providing signals for long and short positions. If a security trades above its highest n periods high, then a long is established. If it trades below its lowest n periods low, then a short is established

(https://en.wikipedia.org/wiki/Donchian_channel)
#### Volume
The volume is indicated by bars of either gray or red colour. The respective value is availabe on the left part of the plot along the y-axis. Exact values for volume cannot be extracted currently. A red bar indicates a decrease in stock price over the time period. A grey bar indicates an increase in stock price over the given time period.
#### Close price line
The black line represents the close price line of the stock over a period of 48months, dates are visible at bottom of graph extracted from x value.

 *If you would like a candle stick pattern please add `style="candlestick"` to the arguments of the arguments of the plot command*
 
The default candlestick patterns are available over the course of the day, these follow the typical format with grey representing a rise in prices and red representing a fall in prices. Exact price and date values of cursor position are available at the bottom. The x-value represents date, while the y-value represents the price at cursor location.
#### Triangles
The triangles represent positions taken up according to the strategy mentioned by code above. A buy position/ signal will be generated if the triangle is green and pointed upwards, while a sell position/ sell signal is generated with the red downward pointing triangle.
#### Trading dots
The next canvas involves a set of dots representing either profit or losses which have occured by following the strategy. Here too the magnitudes can be separated by the colour of the dot and the cursor value on hovering over dots. One can also approximate the values with the data on the scale located on the right.
#### Portfolio value
The final screen on the top indicates the portfolio value and cash value. Each are differentiated by the respective color.

In [ ]:
# Define Symbol list (symbols_02) (Current)
symbols_02 = "SIEMENS,ASIANPAINT,SBICARD,INFY,BRITANNIA,ULTRACEMCO,HINDUNILVR,LT"

In [ ]:
# Deine Symbol list (symbols_03)
symbols_03 = "LICI,COALINDIA,EASEMYTRIP,IEX,CGPOWER,MSUMI,BEPL,IRCTC,CHEMPLASTS,CHENNPETRO,MARICO,TTML,NMDC,NATIONALUM,ITC,FACT,GNFC,VEDL,LSIL,GRAVITA,GUJGASLTD,KPITTECH,RITES,UGARSUGAR,JSWSTEEL,MADRASFERT,FINPIPE,BEL,PETRONET,ASHOKA,MAZDOCK,ENGINERSIN,DATAMATICS,JAMNAAUTO,MOREPENLAB,MARKSANS,ZENTEC,SAIL,CROMPTON,DABUR,CHAMBLFERT,TRIDENT,GAIL,SPIC,NBCC,SONACOMS,BLS,LAURUSLABS,TRITURBINE,MIRZAINT"

In [ ]:
# Define Symbol list (symbols_04)
symbols_04 = "IRFC,RVNL,ZOMATO,MANAPPURAM,IRCON,BHEL,ENGINERSIN,PNB,IDFCFIRSTB,TATASTEEL,PATELENG,FEDERALBNK,MAHABANK,BANKBARODA,HUDCO,BANKINDIA,ICICIBANK,J&KBANK,UCOBANK,SJVN,NBCC,SAIL,CENTRALBK,UNIONBANK,SBIN,NHPC,IOB,NYKAA,BEL,AXISBANK,HSCL,TATAMOTORS,M&MFIN,CANBK,RECLTD,UJJIVANSFB,IOC,JISLJALEQS,ADANIPOWER,TATAPOWER,RENUKA,IDFC,L&TFH,IBULHSGFIN,AMBUJACEM,EQUITASBNK,GSFC,MOTHERSON,PFC,GMRINFRA,ITC,ONGC,IRB,MMTC,MSUMI,INDHOTEL,RBLBANK,IEX,NATIONALUM,GICRE,NMDC,LEMONTREE,ABFRL,TRIDENT,NIACL,ITI,NTPC,HFCL,PSB,PAYTM,JKTYRE,NFL,RAILTEL,LSIL,GAIL"

In [ ]:
# Download historical data for calculations (2months at 60min)
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "7d"
intervalOfData = "15m"
apiSymbol = []
for symbol in symbols_02.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot_02 = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot_02[apiSymbolIndivi] = stockPandas

In [ ]:
# Define variables for filtering
## Variables
# Number of days checked [Can we push this dates checked to different cell?]
datesChecked = 7
# weights of each individual day (Shape in reshape function should be matched. )
weights = np.array([1, 15, 10, 5, 2])
# Define the strategy values here
DonchianPeriod = 21
MACDShort = 12
MACDLong = 26
MACDSignal = 9
# Adding additional weights, will be helfull only while checking.
weights = np.append(weights, np.ones((datesChecked-weights.size,1)))
signalBullDict = {}

In [ ]:
# Calculate through the stocklist
for symbol_02 in stockPandasTot_02:
    # Reversing data because finta likes calculations with oldest date on top
    selectData = stockPandasTot_02[symbol_02][::-1]
    # These data are reversed again to show the latest date at the top row.
    # The indicator taken is PSAR from finta
    donch = finta.TA.DO(selectData,upper_period=DonchianPeriod,lower_period=DonchianPeriod)
    macd = finta.TA.MACD(selectData,period_fast=MACDShort,period_slow=MACDLong,signal=MACDSignal)
    # Dema is already a series and ready for comparision
    donchUpp = donch['UPPER'][::-1]
    macdHist = (macd['MACD'] - macd['SIGNAL'])[::-1]
    close = selectData['close'][::-1]
    # Comparision being done in this section
    firstSet = (close >= donchUpp) & (close.shift(-1) >= donchUpp.shift(-1))
    secondSet = (macdHist >= macdHist.shift(-1))
    # Comparisions are like this
    signChangeGood = (firstSet & secondSet).astype(int).to_numpy()
    # signChangeBad = ((diff_0 < 0) & (diff_1 > 0)).astype(int).to_numpy()
    signChange = signChangeGood 
    signChange = signChange[0:datesChecked]
    signalBull = weights*signChange
    signalBullDict[symbol_02] = signalBull
# print(psar)

In [ ]:
# Display filter results as dataframe
signalBullPand = pd.DataFrame(signalBullDict).T

def color_mapper(row):
    # compute the sum of the row
    sum = row.sum()
    # if the sum is negative, return red
    if sum < 0:
        return ['color: white; background-color: rgb(120, 50, 20)'] * len(row)
    # if the sum is positive, map it to a shade of green based on its magnitude
    else:
        magnitude = sum*50
        r = 15
        g = magnitude
        b = 15
        return [f'color: white; background-color: rgb({r},{g},{b})'] * len(row)

def make_pretty(styler):
    styler.apply(color_mapper,axis=1)
    styler.format(precision=1)
    return styler

signalBullPand.style.pipe(make_pretty)